## Snetiment Scores Calculation with Dask Parallel Computing

### Li Liu
### May, 2019

In [26]:
from dask import compute, delayed
import dask.multiprocessing
import multiprocessing
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
import pandas as pd
import timeit
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
df = pd.read_csv('CSV_Files_Posts_sample.csv')
df['Body'].iloc[df.shape[0]-1]

C:\Users\lliu9\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,1,2,3,5,6,8,9,15,16,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


"Visual Studio disassembles this to the following: Obviously, a general protection fault happens at 00417002 because eax does not point to anything meaningful, but even if I nop it out (90 90 90) it throws an illegal opcode exception at 00417005 (it does not get handled by the kernel). I'm pretty sure that this is some sort of data and not executable code."

In [32]:
df.shape[0]

374275

In [28]:
def sentiment_analyzer_scores(sentence):
    try:
        score = analyser.polarity_scores(sentence)
        comp = score["compound"]
        return comp
    except AttributeError:
        pass

In [29]:
num_cores = multiprocessing.cpu_count()
num_cores

8

In [ ]:
start_time = timeit.default_timer()
lazy_values = []
for sim in range(df.shape[0]):
    val = delayed(sentiment_analyzer_scores)(df['Body'].iloc[sim])
    lazy_values.append(val)
results_par = compute(*lazy_values, 
                          scheduler=dask.multiprocessing.get, num_workers=num_cores)
elapsed_time = timeit.default_timer() - start_time
print('Elapsed time=', elapsed_time, 'seconds')

In [ ]:
sns.distplot(results_par)